In [ ]:
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report


dataLabels = pd.read_csv('Image_classification_data/data_labels_mainData.csv')
dataLabels['isCancerous'] = dataLabels['isCancerous'].astype(str)

modelToTrain =  "both" # "cellType" # "isCancerous" # 

img_size = (27, 27)
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)


data_dir = 'Image_classification_data/patch_images'  


isCancerous_train_data = datagen.flow_from_dataframe(
    dataframe=dataLabels,
    directory='Image_classification_data/patch_images',
    x_col='ImageName', 
    y_col='isCancerous',  
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training' 
)

cellType_train_data = datagen.flow_from_dataframe(
    dataframe=dataLabels,
    directory='Image_classification_data/patch_images',
    x_col='ImageName', 
    y_col='cellTypeName',  
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training' 
)

isCancerous_val_data = datagen.flow_from_dataframe(
    dataframe=dataLabels,
    directory='Image_classification_data/patch_images',
    x_col='ImageName',  
    y_col='isCancerous',  
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

cellType_val_data = datagen.flow_from_dataframe(
    dataframe=dataLabels,
    directory='Image_classification_data/patch_images',
    x_col='ImageName',  
    y_col='cellTypeName',  
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

dataLabels['isCancerous'].value_counts()

Found 7917 validated image filenames belonging to 2 classes.
Found 7917 validated image filenames belonging to 4 classes.
Found 1979 validated image filenames belonging to 2 classes.
Found 1979 validated image filenames belonging to 4 classes.


isCancerous
0    5817
1    4079
Name: count, dtype: int64

In [169]:
reg_lambda = 0.001


if os.path.exists('isCancerousModel.keras'):
    isCancerousModel = load_model('isCancerousModel.keras')  
    print("isCancerous model loaded successfully.")
else:
    isCancerousModel = Sequential([
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(27, 27, 3)), 
        Conv2D(32, (3, 3), activation='relu', padding='same'), 
        MaxPooling2D(2, 2),
            
        Conv2D(64, (3, 3), padding='same', activation='relu'),
        Conv2D(64, (3, 3), padding='same', activation='relu'),
        MaxPooling2D(2, 2),

        Conv2D(128, (3, 3), padding='same', activation='relu'),
        Conv2D(128, (3, 3), padding='same', activation='relu'),
        MaxPooling2D(2, 2),

        Flatten(),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid') 
    ])
    print("No isCancerous model found, creating new one")


if os.path.exists('cellTypeModel.keras'):
    cellTypeModel = load_model('cellTypeModel.keras')  
    print("cellType model loaded successfully.")
else:
    cellTypeModel = Sequential([
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(27, 27, 3)),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(2, 2),
            
        Conv2D(64, (3, 3), padding='same', activation='relu'),
        Conv2D(64, (3, 3), padding='same', activation='relu'),
        MaxPooling2D(2, 2),

        Flatten(),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dense(4, activation='softmax') 
    ])
    print("No cellType model found, creating new one")


isCancerous model loaded successfully.
cellType model loaded successfully.


In [170]:
if modelToTrain == "isCancerous" or modelToTrain == "both":
    isCancerousModel.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    isCancerousModel.fit(
        isCancerous_train_data,
        validation_data=isCancerous_val_data,
        epochs=50,
        callbacks=[early_stop],
    )
if modelToTrain == "cellType" or modelToTrain == "both":
    cellTypeModel.compile(
    optimizer='adam',
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['categorical_accuracy']
    )

    cellTypeModel.fit(
        cellType_train_data,
        validation_data=cellType_val_data,
        epochs=50,
        callbacks=[early_stop],
    )

Epoch 1/50


/home/leo/cancerAI/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


248/248 ━━━━━━━━━━━━━━━━━━━━ 17s 51ms/step - accuracy: 0.8755 - loss: 0.3111 - val_accuracy: 0.9020 - val_loss: 0.2621
Epoch 2/50
248/248 ━━━━━━━━━━━━━━━━━━━━ 9s 35ms/step - accuracy: 0.8871 - loss: 0.2697 - val_accuracy: 0.8706 - val_loss: 0.2835
Epoch 3/50
248/248 ━━━━━━━━━━━━━━━━━━━━ 8s 34ms/step - accuracy: 0.8872 - loss: 0.2672 - val_accuracy: 0.8868 - val_loss: 0.2686
Epoch 4/50
248/248 ━━━━━━━━━━━━━━━━━━━━ 8s 34ms/step - accuracy: 0.8782 - loss: 0.2959 - val_accuracy: 0.8969 - val_loss: 0.2756
Epoch 5/50
248/248 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - accuracy: 0.8938 - loss: 0.2548 - val_accuracy: 0.8792 - val_loss: 0.2842
Epoch 6/50
248/248 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.8998 - loss: 0.2436 - val_accuracy: 0.8888 - val_loss: 0.2910
Epoch 1/50


/home/leo/cancerAI/lib/python3.12/site-packages/keras/src/backend/tensorflow/nn.py:666: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


248/248 ━━━━━━━━━━━━━━━━━━━━ 16s 51ms/step - categorical_accuracy: 0.7297 - loss: 0.6858 - val_categorical_accuracy: 0.7034 - val_loss: 0.8092
Epoch 2/50
248/248 ━━━━━━━━━━━━━━━━━━━━ 9s 34ms/step - categorical_accuracy: 0.7452 - loss: 0.6493 - val_categorical_accuracy: 0.6700 - val_loss: 0.8464
Epoch 3/50
248/248 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - categorical_accuracy: 0.7548 - loss: 0.6274 - val_categorical_accuracy: 0.7352 - val_loss: 0.7527
Epoch 4/50
248/248 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - categorical_accuracy: 0.7613 - loss: 0.6060 - val_categorical_accuracy: 0.7256 - val_loss: 0.7328
Epoch 5/50
248/248 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - categorical_accuracy: 0.7612 - loss: 0.5938 - val_categorical_accuracy: 0.6862 - val_loss: 0.8658
Epoch 6/50
248/248 ━━━━━━━━━━━━━━━━━━━━ 9s 35ms/step - categorical_accuracy: 0.7538 - loss: 0.6023 - val_categorical_accuracy: 0.6877 - val_loss: 0.8138
Epoch 7/50
248/248 ━━━━━━━━━━━━━━━━━━━━ 9s 38ms/step - categorical_accuracy: 0.7757 - loss: 

In [ ]:
if modelToTrain == "isCancerous" or modelToTrain == "both":
    print("Is cancerous: ")
    isCancerousModel.evaluate(isCancerous_val_data)
    isCancerousModel.save('isCancerousModel.keras')

if modelToTrain == "cellType" or modelToTrain == "both":
    print("Cell type: ")
    cellTypeModel.evaluate(cellType_val_data)
    cellTypeModel.save('cellTypeModel.keras')

y_true = isCancerous_val_data.classes
y_pred = (isCancerousModel.predict(isCancerous_val_data) > 0.5).astype(int)

print(classification_report(y_true, y_pred))

Is cancerous: 
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.9050 - loss: 0.2717
Cell type: 
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - categorical_accuracy: 0.7350 - loss: 0.7153
